In [1]:
import itertools
import numpy as np
from enum import IntEnum
from tqdm import tqdm
from collections import Counter

class Dir(IntEnum):
    U = 0
    R = 1
    D = 2
    L = 3

class Turn(IntEnum):
    LEFT = 3
    STRAIGHT = 0
    RIGHT = 1

IDX_TO_PIECE = {
    (Dir.U, Dir.R, Dir.D, Dir.L): "+",
    (Dir.R, Dir.L): "-",
    (Dir.U, Dir.D): "|",
    (Dir.U, Dir.R): "\\",
    (Dir.D, Dir.L): "\\",
    (Dir.U, Dir.L): "/",
    (Dir.R, Dir.D): "/",
}

class Cart:
    UNDER = {'>': '-', '<': '-', '^': '|', 'v': '|'}
    DIRS = {'^': Dir.U, '>': Dir.R, 'v': Dir.D,  '<': Dir.L}

    def __init__(self, piece, location):
        self.direction = self.DIRS[piece]
        self.location = location
        self.turn_decisions = itertools.cycle([
            Turn.LEFT,
            Turn.STRAIGHT,
            Turn.RIGHT
        ])
    
    def __repr__(self):
        return f"{self.piece}@{self.xy_coordinate}"
    
    def __lt__(self, other):
        return self.location < other.location
    
    def __eq__(self, other):
        return self.location == other.location
        
    @property
    def piece(self):
        dir_to_piece = {v: k for k, v in self.DIRS.items()}
        return dir_to_piece[self.direction]
    
    @property
    def xy_coordinate(self):
        y, x = self.location
        return f'{x},{y}'
    
    def maybe_turn(self, track_underneath):
        # by default, do nothing
        turn = Turn.STRAIGHT
        
        # if we're at an intersection, check to see next turn
        if track_underneath == "+":
            turn = next(self.turn_decisions)
            
        # if we're at a curve, adjust as necessary
        elif track_underneath == "/":
            if self.direction in (Dir.U, Dir.D):
                turn = Turn.RIGHT
            else:
                turn = Turn.LEFT
        elif track_underneath == "\\":
            if self.direction in (Dir.U, Dir.D):
                turn = Turn.LEFT
            else:
                turn = Turn.RIGHT
        # print(f"{self}: turning={turn} track={track_underneath}")
        self.direction = Dir((self.direction + turn) % 4)
        
    def step(self, track_array):
        track_underneath = track_array[self.location]
        adjacent = get_adjacent(self.location, track_array)
        self.maybe_turn(track_underneath)
        self.location = adjacent[self.direction]

def get_adjacent(location, area):
    row, col = location
    m, n = area.shape
    left = (row, col - 1) if col > 0 else None
    right = (row,  col + 1) if col < (n - 1) else None
    up = (row - 1, col) if row > 0 else None
    down = (row + 1, col) if row < (m - 1) else None
    return up, right, down, left

def parse(text):
    area = np.array([[c for c in line] for line in text.splitlines()])
    carts = []
    nodes = {}
    for row, col in np.argwhere(area != ' '):
        location = (row, col)
        node = {}
        up, right, down, left = get_adjacent(location, area)
        piece = area[row, col]

        # deal with carts and put back an ordinary track piece
        if piece in Cart.UNDER:
            cart = Cart(piece, location)
            carts.append(cart)
            piece = Cart.UNDER[piece]

        # deal with straights and intersections
        if piece in '+-':
            node[Dir.L], node[Dir.R] = left, right
        if piece in '+|':
            node[Dir.U], node[Dir.D] = up, down

        # deal with curves
        m, n = area.shape
        if piece == "\\":
            if up is not None and area[up] in "|+":
                node[Dir.R], node[Dir.U] = right, up
            else:
                node[Dir.L], node[Dir.D] = left, down
        if piece == "/":
            if up is not None and area[up] in "|+":
                node[Dir.L], node[Dir.U] = left, up
            else:
                node[Dir.R], node[Dir.D] = right, down

        nodes[location] = node
    return carts, nodes
            
def to_array(nodes):
    m, n = max(k[0] for k in nodes.keys()) + 1, max(k[1] for k in nodes.keys()) + 1
    arr = np.array([[" " for _ in range(n)] for _ in range(m)], dtype="<U1")
    for (row, col), node in nodes.items():
        exits = tuple(sorted(node.keys()))
        if len(node) >= 3:
            piece = "+"
        else:
            piece = IDX_TO_PIECE[exits]
        arr[row, col] = piece
    return arr

def to_string(nodes, carts=None):
    if carts is None:
        carts = []
    arr = to_array(nodes)
    for cart in carts:
        arr[cart.location] = cart.piece
    return '\n'.join([''.join(row) for row in arr])

def check_crash(carts):
    location_counts = Counter([cart.location for cart in carts])
    for location, n in location_counts.items():
        if n > 1:
            return location
    return None

def play(carts, nodes, max_tick=np.inf, hard_crash=True):
    tick = 0
    track_array = to_array(nodes)
    while tick < max_tick:
        curr = []
        for cart in sorted(carts):
            cart.step(track_array)
            crash_location = check_crash(carts)
            if crash_location is not None:
                carts = [cart for cart in carts if cart.location != crash_location]
                print(f"CRASH! t={tick}: X,Y={cart.xy_coordinate} [remaining={carts}]")
                if hard_crash:
                    return

        if len(carts) < 2:
            yield carts
            return
        
        yield carts
        tick += 1


test_input = r"""/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """
test_carts, test_nodes = parse(test_input)
track = to_array(test_nodes)
assert to_string(test_nodes, carts=test_carts) == test_input
print(to_string(test_nodes))
print(to_string(test_nodes, carts=test_carts))

/---\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   


In [2]:
game = play(test_carts, test_nodes, hard_crash=False)
while True:
    try:
        print(to_string(test_nodes, carts=next(game)))
    except StopIteration:
        break

/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-v--/
  \------/   
/--->        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   
/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   
/---\        
|   |  /----\
| /-v--+-\  |
| | |  | |  |
\-+-/  \-+-->
  \------/   
/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   
/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   
/---\        
|   |  /----^
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
/---\        
|   |  /--<-\
| /-+--+->  |
| | |  | |  |
\-+-/  \-+--/
  \------/   
/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
/---\        
|   |  /<---\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-v--/
  \------/   
/---\        
|   |  <----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \<+--/
  \---

### Part 1

In [3]:
with open("../inputs/13/input.txt") as fp:
    carts, nodes = parse(fp.read())
game = play(carts, nodes)
while True:
    try:
        next(game)
    except StopIteration:
        break

CRASH! t=144: X,Y=136,36 [remaining=[v@130,93, >@26,73, >@28,34, >@94,100, v@123,24, >@64,0, v@131,118, >@106,12, ^@83,67, <@82,61, ^@121,65, v@145,91, <@58,85, ^@41,137, <@112,144]]


### Part 2

In [4]:
with open("../inputs/13/input.txt") as fp:
    carts, nodes = parse(fp.read())
game = play(carts, nodes, hard_crash=False)
while True:
    try:
        remaining = next(game)
    except StopIteration:
        break

last = remaining[0]
# one more tick...
last.step(to_array(nodes))
last.xy_coordinate

CRASH! t=144: X,Y=136,36 [remaining=[v@130,93, >@26,73, >@28,34, >@94,100, v@123,24, >@64,0, v@131,118, >@106,12, ^@83,67, <@82,61, ^@121,65, v@145,91, <@58,85, ^@41,137, <@112,144]]
CRASH! t=205: X,Y=98,38 [remaining=[v@105,121, >@66,94, ^@65,26, >@140,115, ^@132,12, >@125,0, v@144,130, <@125,22, v@40,54, <@139,144, v@25,79, <@7,110, <@127,114]]
CRASH! t=220: X,Y=137,3 [remaining=[>@108,127, v@68,101, ^@73,20, v@137,127, <@142,143, <@120,31, v@36,65, <@125,143, v@25,90, ^@4,98, <@116,110]]
CRASH! t=478: X,Y=133,69 [remaining=[>@85,65, ^@48,42, >@12,71, v@63,113, >@18,18, <@42,36, >@28,78, ^@87,78, v@130,63]]
CRASH! t=988: X,Y=41,103 [remaining=[^@133,39, v@121,46, v@9,69, >@69,9, <@74,93, ^@22,34, <@49,18]]
CRASH! t=2250: X,Y=117,106 [remaining=[<@70,79, >@79,124, ^@144,107, v@81,46, >@77,21]]
CRASH! t=7128: X,Y=93,49 [remaining=[v@42,79, >@46,21, ^@110,53]]
CRASH! t=12618: X,Y=53,63 [remaining=[<@53,112]]


'53,110'